<a href="https://colab.research.google.com/github/TuragDev/ML-Python-Codes/blob/main/crystal_system_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
!pip install xgboost

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving Orbital_mp_dataset.csv to Orbital_mp_dataset.csv
Saving sine_mp_data.csv to sine_mp_data.csv
Saving Xrd_mp_dataset.csv to Xrd_mp_dataset.csv


In [ ]:
import joblib
from sklearn.model_selection import train_test_split
import numpy as np

import pandas as pd

from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

import numpy as np

import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import sys
# In[3]:


from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV


Xrd Descriptor

In [ ]:
df= pd.read_csv('Xrd_mp_dataset.csv')

In [ ]:
X=df.iloc[:,14:]
Y=df['crystal_system']

In [ ]:
from sklearn.preprocessing import StandardScaler
# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X)

# apply the scaler to both the training and test data
X_normalized = scaler.transform(X)

In [ ]:
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_encoded = label_encoder.fit_transform(Y)

In [ ]:
import xgboost
from hyperopt import fmin, tpe, hp

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=108)

In [ ]:
# Define the search space for hyperparameters
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

# Define the objective function to minimize (negative accuracy)
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [06:18<00:00,  7.57s/trial, best loss: -0.6912751677852349]
Best Hyperparameters: {'colsample_bytree': 0.8898723070401753, 'gamma': 0.013321041374090736, 'learning_rate': 1.798447472004195, 'max_depth': 9.0, 'min_child_weight': 2.0, 'n_estimators': 172.0, 'reg_alpha': 2.0268486003203305, 'reg_lambda': 3.034063856036684, 'subsample': 0.9737162445108123}


In [14]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 172,
    'learning_rate':  1.798447472004195,
    'gamma': 0.013321041374090736,
    'max_depth': 9,
    'min_child_weight': 2,
    'colsample_bytree': 0.8898723070401753,
    'subsample':  0.9737162445108123,
    'reg_alpha': 2.026848600320330,
    'reg_lambda': 3.034063856036684
}

# Create an XGBoost classifier with the best hyperparameters
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)

Mean Accuracy: 0.6830826330532211


In [15]:
model.fit(X_train, y_train)
pred = model.predict(X_train)
training_accuracy= accuracy_score(y_train,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test)
test_accuracy= accuracy_score(y_test,pred)
print("Accuracy on Test data: ",test_accuracy)
xrd_Bayes_crystal = label_encoder.inverse_transform(pred)
xrd_Bayes_crystal

Accuracy on Training data:  0.9983221476510067
Accuracy on Test data:  0.6308724832214765


array(['trigonal', 'triclinic', 'cubic', 'triclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'monoclinic', 'monoclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'hexagonal', 'triclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'monoclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'triclinic', 'trigonal',
       'monoclinic', 'cubic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'monoclinic', 'triclinic', 'monoclinic', 'monoclinic',
       'triclinic', 'monoclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'cubic',
       'triclinic', 'monoclinic', 'triclinic', 'orthorhombic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'orthorhombic', 'triclinic', 'triclinic',
       'monoclinic', 'cubic', 'hexagonal', 'triclinic', 'trigonal',
       'monoclinic', 'tricl

SCM descriptor

In [17]:
df= pd.read_csv('sine_mp_data.csv')

In [18]:
X=df.iloc[:,14:]
Y=df['crystal_system']

In [19]:
from sklearn.preprocessing import StandardScaler
# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X)

# apply the scaler to both the training and test data
X_normalized = scaler.transform(X)

In [20]:
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_encoded = label_encoder.fit_transform(Y)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=108)

In [22]:
# Define the search space for hyperparameters
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

# Define the objective function to minimize (negative accuracy)
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [02:48<00:00,  3.38s/trial, best loss: -0.6845637583892618]
Best Hyperparameters: {'colsample_bytree': 0.7843941419008446, 'gamma': 4.437924434568651, 'learning_rate': 1.44761690016674, 'max_depth': 6.0, 'min_child_weight': 4.0, 'n_estimators': 38.0, 'reg_alpha': 0.3528037379168347, 'reg_lambda': 4.433746354678788, 'subsample': 0.9931218851055572}


In [23]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 38,
    'learning_rate': 1.44761690016674,
    'gamma':  4.437924434568651,
    'max_depth': 6,
    'min_child_weight': 4,
    'colsample_bytree':  0.7843941419008446,
    'subsample': 0.9931218851055572,
    'reg_alpha':0.3528037379168347,
    'reg_lambda': 4.433746354678788
}

# Create an XGBoost classifier with the best hyperparameters
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)

Mean Accuracy: 0.6934439775910364


In [24]:
model.fit(X_train, y_train)
pred = model.predict(X_train)
training_accuracy= accuracy_score(y_train,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test)
test_accuracy= accuracy_score(y_test,pred)
print("Accuracy on Test data: ",test_accuracy)
Sine_Bayes_crystal = label_encoder.inverse_transform(pred)
Sine_Bayes_crystal

Accuracy on Training data:  0.7953020134228188
Accuracy on Test data:  0.6711409395973155


array(['cubic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'cubic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'cubic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'cubic', 'triclinic',
       'monoclinic', 'triclinic', 'orthorhombic', 'triclinic',
       'monoclinic', 'monoclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'monoclinic', 'triclinic', 'triclinic', 'cubic',
       'monoclinic', 'triclinic', 'monoclinic', 'monoclinic', 'triclinic',
  

OFM Descriptor

In [25]:
df= pd.read_csv('Orbital_mp_dataset.csv')

In [26]:
X=df.iloc[:,14:]
Y=df['crystal_system']

In [27]:
from sklearn.preprocessing import StandardScaler
# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X)

# apply the scaler to both the training and test data
X_normalized = scaler.transform(X)

In [28]:
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_encoded = label_encoder.fit_transform(Y)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=108)

In [30]:
# Define the search space for hyperparameters
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

# Define the objective function to minimize (negative accuracy)
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [13:28<00:00, 16.17s/trial, best loss: -0.6778523489932886]
Best Hyperparameters: {'colsample_bytree': 0.9898377591466134, 'gamma': 3.2449384628001896, 'learning_rate': 1.0414770546861243, 'max_depth': 7.0, 'min_child_weight': 2.0, 'n_estimators': 294.0, 'reg_alpha': 2.264100689692387, 'reg_lambda': 0.018928928896292163, 'subsample': 0.9206812497078292}


In [33]:
import xgboost
from xgboost import XGBClassifier

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 294,
    'learning_rate': 1.0414770546861243,
    'gamma': 3.2449384628001896,
    'max_depth': 7,
    'min_child_weight': 2,
    'colsample_bytree': 0.9898377591466134,
     'subsample': 0.9206812497078292,
    'reg_alpha':  2.264100689692387,
    'reg_lambda': 0.018928928896292163
}

# Create an XGBoost classifier with the best hyperparameters
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)

Mean Accuracy: 0.6512605042016806


In [34]:
model.fit(X_train, y_train)
pred = model.predict(X_train)
training_accuracy= accuracy_score(y_train,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test)
test_accuracy= accuracy_score(y_test,pred)
print("Accuracy on Test data: ",test_accuracy)
Orbital_Bayes_crystal = label_encoder.inverse_transform(pred)
Orbital_Bayes_crystal

Accuracy on Training data:  0.7277310924369748
Accuracy on Test data:  0.6510067114093959


array(['cubic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'cubic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'cubic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'monoclinic', 'triclinic', 'triclinic', 'monoclinic',
       'monoclinic', 'monoclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'orthorhombic',
